In [1]:
import tensorflow as tf
from tensorflow import keras

In [10]:
(xfull,yfull),(xtest,ytest)=keras.datasets.fashion_mnist.load_data()
xvalid,xtrain=xfull[:5000],xfull[5000:]
yvalid,ytrain=yfull[:5000],yfull[5000:]
validset=tf.data.Dataset.from_tensor_slices((xvalid,yvalid))
trainset=tf.data.Dataset.from_tensor_slices((xtrain,ytrain)).shuffle(len(xtrain))
testset=tf.data.Dataset.from_tensor_slices((xtest,ytest))

In [13]:
from tensorflow.train import Feature,Features,Example,BytesList
trainexam=Example(
    features=Features(
        feature={
            'a'=tf.io.serialize_tensor(trainset),
            'b'
        }
    )
)


In [14]:
trainset[0]

TypeError: 'ShuffleDataset' object is not subscriptable